# AI Manga Colab Post-Production Toolset



> You can omit any script you want. Run the scripts you need consecutively (in the order they're listed; this is important!).


This script renames all the manga pages to "1.png 2.png 3.png", "01.png 02.png 03.png", or "001.png 002.png 003.png" depending on the amount of the files present in each folder.

In [ ]:
# @title Page Renamer {display-mode: "form"}

import os
from google.colab import drive
from tqdm import tqdm

def check_connect_gdrive():
  if not os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive connection in progress...")
    drive.mount('/content/gdrive')

def rename_pages(path):
  for folder in tqdm(os.listdir(path), desc="Renaming", unit="dir"):
    folder_path = os.path.join(path, folder)

    if os.path.isdir(folder_path):
      # This function renames files in a format that ensures a consistent number of digits,
      # adding leading zeros based on the total number of files: 1 digit for less than 10 files,
      # 2 digits for 10 to 99 files, and 3 digits for 100 or more files. So, digit_number represents
      # the minumum amount of characters a name will hold -- by default just 1.
      file_count = 0
      digit_number = 1

      # Count the amount of files in a dir; rename files to temp names, so that the possibility of names colliding is accounted for.
      for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)

        if os.path.isfile(file_path):
            file_count += 1
            new_file_path = os.path.join(folder_path, "tmpnm_" + file)

            if file_path != new_file_path:
                os.rename(file_path, new_file_path)

      # Calculate the minimal amount of characters needed based on the amount of files.
      if (file_count > 9 and file_count < 100):
        digit_number = 2
      elif (file_count >= 100):
        digit_number = 3

      file_number = 0
      for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)

        if os.path.isfile(file_path):
            file_number += 1
            new_file_path = os.path.join(folder_path, '{:0{}}'.format(file_number, digit_number) + os.path.splitext(file)[1])

            if file_path != new_file_path:
                os.rename(file_path, new_file_path)

def main():
  print("[AI Manga Upscale Colab] Page Renamer")

  check_connect_gdrive()

  if os.path.exists("/content/gdrive/MyDrive/ESRGAN/output"):
    rename_pages("/content/gdrive/MyDrive/ESRGAN/output")

if __name__ == "__main__":
  main()

Use imagemagick to downscale your result. Just remember to have enough space on your Google Drive.

In [ ]:
# @title Downscaler {display-mode: "form"}

# @markdown <h1>Select a target resolution or a percentage to downscale by</h1>
selection = "2145x3056" # @param ["2145x3056", "75%", "50%", "custom"]
# @markdown <h1>This will be ignored unless "custom" is selected in the previous field. Provide either a percentage or a resolution below.
custom_selection = "" # @param {type:"string"}

import os
import subprocess
from google.colab import drive
from tqdm import tqdm

def check_install_imagemagick():
  if not os.path.exists('/usr/bin/mogrify'):
    print("Installing imagemagick...")
    subprocess.run(["apt", "-y", "install", "imagemagick"])

def check_connect_gdrive():
  if not os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive connection in progress...")
    drive.mount('/content/gdrive')

def check_create_downscaled():
  if not os.path.exists('/content/gdrive/MyDrive/ESRGAN/output_downscaled'):
    os.mkdir('/content/gdrive/MyDrive/ESRGAN/output_downscaled')

def downscale(path, resize_size):
  if not os.path.isdir(path):
    print(f"{path} is not a valid directory. Cannot continue.")
    return

  files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
  path_out = os.path.join(path, "_scaled")

  if not os.path.exists(path_out):
    os.mkdir(path_out)

  for file in files:
      file_path = os.path.join(path, file)
      subprocess.run(["mogrify", "-path", path_out, "-resize", resize_size, "-format", "png", file_path])

def move_delete_folder(source_folder, destination_folder):
    if not os.path.exists(source_folder):
        print(f"The source folder '{source_folder}' does not exist.")
        return

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    files = os.listdir(source_folder)

    for file in files:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        os.rename(source_path, destination_path)

    os.rmdir(source_folder)

def loop_downscale(path, resize_size):
  folders = os.listdir(path)

  for folder in tqdm(folders, desc="Downscaling", unit="dir"):
      folder_path = os.path.join(path, folder)

      if os.path.isdir(folder_path):
          downscale(folder_path, resize_size)
          move_delete_folder(os.path.join(folder_path, "_scaled"), os.path.join('/content/gdrive/MyDrive/ESRGAN/output_downscaled', folder))

def main():
  print("[AI Manga Upscale Colab] Downscaler")

  check_install_imagemagick()
  check_connect_gdrive()
  check_create_downscaled()

  if selection != "custom":
    print(f"Using {selection}")
    loop_downscale("/content/gdrive/MyDrive/ESRGAN/output", selection)
  else:
    print(f"Using {custom_selection}")
    loop_downscale("/content/gdrive/MyDrive/ESRGAN/output", custom_selection)

if __name__ == "__main__":
  main()

Compile the result into the CBZ files.

In [ ]:
# @title CBZ Creator {display-mode: "form"}

import os
import zipfile
from tqdm import tqdm

def check_connect_gdrive():
  if not os.path.exists('/content/gdrive/MyDrive'):
    print("Google Drive connection in progress...")
    drive.mount('/content/gdrive')

def cbz_files(path, output_path):
    for folder in tqdm(os.listdir(path), desc="Creating CBZs", unit="cbz"):
        folder_path = os.path.join(path, folder)

        if os.path.isdir(folder_path):
            cbz_path = os.path.join(output_path, folder + ".cbz")

            with zipfile.ZipFile(cbz_path, 'w', zipfile.ZIP_DEFLATED) as cbz_file:
                for file in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file)

                    if os.path.isfile(file_path):
                        # We don't want to include the folder into the cbz file's structure, so
                        # we calculate the relative path and use it instead.
                        relative_path = os.path.relpath(file_path, folder_path)
                        cbz_file.write(file_path, arcname=relative_path)

def main():
  print("[AI Manga Upscale Colab] CBZ Creator")

  check_connect_gdrive()

  if not os.path.exists("/content/gdrive/MyDrive/ESRGAN/output_cbz"):
    os.mkdir("/content/gdrive/MyDrive/ESRGAN/output_cbz")

  if os.path.exists("/content/gdrive/MyDrive/ESRGAN/output_downscaled"):
    cbz_files("/content/gdrive/MyDrive/ESRGAN/output_downscaled", "/content/gdrive/MyDrive/ESRGAN/output_cbz")
  elif os.path.exists("/content/gdrive/MyDrive/ESRGAN/output"):
    cbz_files("/content/gdrive/MyDrive/ESRGAN/output", "/content/gdrive/MyDrive/ESRGAN/output_cbz")


if __name__ == "__main__":
  main()